In [21]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../data')
from data_loading import *

from Model import *

fname = "../data/up_down/100GEV-downquark-seed1.txt"

jets = jets_from_pythia_txt(fname)
print("Number of jets in file: ", len(jets))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Number of jets in file:  10000


In [25]:
from JetImages import JetImage

image = JetImage(jets[0])

list(image.sort_particles_into_bins())

-1.6934036537
[-1.3190739  -1.30491637 -1.67354666 -1.36750099 -1.41400288 -1.78856034
 -1.6715068  -1.69215943 -1.7530369  -1.57908338 -1.80660624 -1.61700943
 -1.73712869 -1.74766275 -1.66844469 -1.66365173 -1.67172273 -1.71858522
 -1.69085833 -1.70219252 -1.69272442 -1.70647378]
[-0.8  -0.75 -0.7  -0.65 -0.6  -0.55 -0.5  -0.45 -0.4  -0.35 -0.3  -0.25
 -0.2  -0.15 -0.1  -0.05  0.    0.05  0.1   0.15  0.2   0.25  0.3   0.35
  0.4   0.45  0.5   0.55  0.6   0.65  0.7   0.75  0.8 ]
[-2.49340365 -2.44340365 -2.39340365 -2.34340365 -2.29340365 -2.24340365
 -2.19340365 -2.14340365 -2.09340365 -2.04340365 -1.99340365 -1.94340365
 -1.89340365 -1.84340365 -1.79340365 -1.74340365 -1.69340365 -1.64340365
 -1.59340365 -1.54340365 -1.49340365 -1.44340365 -1.39340365 -1.34340365
 -1.29340365 -1.24340365 -1.19340365 -1.14340365 -1.09340365 -1.04340365
 -0.99340365 -0.94340365 -0.89340365]


[(24, 14),
 (24, 16),
 (17, 9),
 (23, 15),
 (22, 18),
 (15, 11),
 (17, 12),
 (17, 12),
 (15, 21),
 (19, 18),
 (14, 18),
 (18, 17),
 (16, 17),
 (15, 17),
 (17, 17),
 (17, 17),
 (17, 17),
 (16, 17),
 (17, 17),
 (16, 17),
 (17, 17),
 (16, 16)]